In [1]:
import numpy
import pandas
import tensorflow as tf
 
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

models =  tf.keras.models
layers = tf.keras.layers 
scikit_learn = tf.keras.wrappers.scikit_learn



/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
Sequential = models.Sequential
Dense = layers.Dense
KerasClassifier = scikit_learn.KerasClassifier
to_categorical = tf.keras.utils.to_categorical

In [3]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [4]:

def load_data():
    dataframe = pandas.read_csv("sonar.csv", header=None)
    dataset = dataframe.values
    # split into input (X) and output (Y) variables
    X = dataset[:,0:60].astype(float)
    Y = dataset[:,60]
    le = LabelEncoder()
    le.fit(Y)
    encoded_Y = le.transform(Y)
    return (X, Y, encoded_Y)
    

In [5]:
def create_baseline():
    
    model = Sequential()
    model.add(layers.Dense(2, activation='sigmoid', input_shape=(60, )))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
def do_estimator():
# evaluate model with standardized dataset
    estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
    print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [7]:
def do_normalize():
    numpy.random.seed(seed)
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
    print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [8]:
(X, Y, encoded_Y) = load_data()
do_estimator()

Results: 77.37% (6.96%)


In [9]:
(X, Y, encoded_Y) = load_data()
do_normalize()


Standardized: 78.83% (10.35%)


In [10]:
def create_smaller():
    model = Sequential()
    
    model.add(layers.Dense(30, activation='relu', input_shape=(60, )))
    model.add(layers.Dense(2, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
    return model
    return model

In [11]:
def do_normalize_smaller():
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
    pipeline = Pipeline(estimators)
    kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
    results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
    print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

In [12]:
(X, Y, encoded_Y) = load_data()
do_normalize_smaller()

Smaller: 83.66% (6.82%)
